## 📝 Lab #2-3 (종합 과제): 로지스틱 회귀 vs. K-NN 모델 비교 및 ROC 커브 시각화 (정답)

In [ ]:
# --- 기본 라이브러리 및 데이터 준비 (튜토리얼 본문 코드 재사용) ---
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

path = './WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.drop('customerID', axis=1, inplace=True)
X = df.drop('Churn', axis=1)
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features), ('cat', categorical_transformer, categorical_features)])

### [문제 1] 두 모델 학습 및 성능 평가 (정답)

In [ ]:
# 로지스틱 회귀 파이프라인 생성 및 학습
lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression(solver='liblinear', random_state=42))])
lr_pipeline.fit(X_train, y_train)

# K-NN(k=11) 파이프라인 생성 및 학습
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier(n_neighbors=11))])
knn_pipeline.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 확률 계산 (ROC-AUC 계산용)
y_pred_proba_lr = lr_pipeline.predict_proba(X_test)[:, 1]
y_pred_proba_knn = knn_pipeline.predict_proba(X_test)[:, 1]

# 각 모델의 정확도와 ROC-AUC 점수 계산 및 출력
acc_lr = accuracy_score(y_test, lr_pipeline.predict(X_test))
auc_lr = roc_auc_score(y_test, y_pred_proba_lr)
acc_knn = accuracy_score(y_test, knn_pipeline.predict(X_test))
auc_knn = roc_auc_score(y_test, y_pred_proba_knn)

print(f"로지스틱 회귀 정확도: {acc_lr:.4f}, ROC-AUC: {auc_lr:.4f}")
print(f"K-NN (k=11) 정확도: {acc_knn:.4f}, ROC-AUC: {auc_knn:.4f}")

### [문제 2] ROC 커브 시각화 (정답)

In [ ]:
# 각 모델의 ROC 커브 계산
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_proba_lr)
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_pred_proba_knn)

# Plotly를 사용하여 ROC 커브 시각화
fig = go.Figure()

# 로지스틱 회귀 ROC 커브 추가
fig.add_trace(go.Scatter(x=fpr_lr, y=tpr_lr, mode='lines', name=f'Logistic Regression (AUC={auc_lr:.4f})'))

# K-NN ROC 커브 추가
fig.add_trace(go.Scatter(x=fpr_knn, y=tpr_knn, mode='lines', name=f'K-NN, k=11 (AUC={auc_knn:.4f})'))

# 램덤 추측선 (y=x) 추가
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Guess', line=dict(dash='dash')))

fig.update_layout(
    title='로지스틱 회귀 vs. K-NN ROC 커브',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=600
)
fig.show()

### [문제 3] 결과 분석 (정답 예시)

1. **정확도(Accuracy)** 측면에서는 **로지스틱 회귀** 모델(약 0.8034)이 K-NN 모델(약 0.7743)보다 더 우수한 성능을 보입니다.

2. **ROC-AUC 점수**와 **ROC 커브**를 종합적으로 고려했을 때도 **로지스틱 회귀** 모델이 더 우수합니다. 로지스틱 회귀의 AUC 점수(약 0.8468)가 K-NN(약 0.8122)보다 높으며, 그래프에서도 로지스틱 회귀의 ROC 커브가 K-NN의 커브보다 더 **왼쪽 위 모서리**에 가깝게 그려집니다. 이는 로지스틱 회귀가 모든 임계값(threshold) 수준에서 전반적으로 '진짜 이탈 고객(True Positive)'을 '가짜 이탈 고객(False Positive)'보다 더 잘 구분해낸다는 것을 의미합니다. 따라서 이 데이터셋에서는 로지스틱 회귀가 이탈 고객을 예측하는 데 더 적합한 모델이라고 결론 내릴 수 있습니다.